<a href="https://colab.research.google.com/github/filipe-research/tutorial_noisylabels/blob/main/codes/tutorial_sibgrapi20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>